In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn import preprocessing

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y
        
  def __getitem__(self, index):
    return self.X[index], self.y[index]
    
  def __len__(self):
    return len(self.X)

In [ ]:
class LSTM(nn.Module):
  def __init__(self, num_classes, num_layers, input_size, hidden_size, device):
    super().__init__()
    self.num_classes = num_classes
    self.num_layer = num_layers
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.device = device
    
    self.lstm = nn.LSTM(
      input_size=input_size,
      hidden_size=hidden_size,
      num_layers=num_layers,
      batch_first=True,
      dropout=0.2)
    self.fc_1 = nn.Linear(hidden_size, 128)
    self.fc_2 = nn.Linear(128, num_classes)
    self.relu = nn.ReLU(inplace=True)
    
  def forward(self, x):
    h_0 = torch.zeros(self.num_layer, x.size(0), self.hidden_size, device=self.device)
    c_0 = torch.zeros(self.num_layer, x.size(0), self.hidden_size, device=self.device)
    
    output, (hn, cn) = self.lstm(x, (h_0, c_0))
    hn = hn.view(-1, self.hidden_size)
    out = self.relu(hn)
    out = self.fc_1(out)
    out = self.relu(out)
    out = self.fc_2(out)
    return out

In [ ]:

class Trainer:
  def __init__(self, path):
    self.path = path
    self.n_step_in = 24 * 2
    self.n_step_out = 24 * 3
    
    self.n_epochs = 100
    self.lr = 0.001
    self.input_size = 6
    self.hidden_size = 8
    self.num_layers = 1
    self.num_classes = 24*3
    
    self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
  def set_up(self):
    df = pd.read_csv("./dataset/CUSTOM_v0/공주.csv")
    df.drop(columns=["연도", "일시", "측정소"], inplace=True)

    scaler = preprocessing.MinMaxScaler()
    scaled_np = scaler.fit_transform(df.values)
    scaled_df = pd.DataFrame(scaled_np, columns=df.columns)
    X, y = self._split_sequences(
      input_seq=scaled_df.values,
      output_seq=scaled_df["PM2.5"].values,
      n_step_in=self.n_step_in,
      n_step_out=self.n_step_out)
    
    X_train = X[:-3000]
    X_val = X[-3000:]
    
    y_train = y[:-3000]
    y_val = y[-3000:]
    
    train_dataset = CustomDataset(X_train, y_train)
    val_dataset = CustomDataset(X_val, y_val)

    self.train_dataloader = DataLoader(
      dataset=train_dataset,
      shuffle=True,
      batch_size=128,
    )
    
    self.val_dataloader = DataLoader(
      dataset=val_dataset,
      shuffle=False,
      batch_size=256,
    )
    
  def _split_sequences(self, input_seq, output_seq, n_step_in, n_step_out):
    X, y = list(), list()
    for i in range(len(input_seq)):
      end_idx = i + n_step_in
      out_end_idx = end_idx + n_step_out
      if out_end_idx > len(input_seq):
        break
      seq_x, seq_y = input_seq[i:end_idx], output_seq[end_idx:out_end_idx]
      X.append(seq_x)
      y.append(seq_y)
    return np.array(X), np.array(y)
  

  def train(self):
    model = LSTM(
      num_classes=self.num_classes,
      num_layers=self.num_layers,
      input_size=self.input_size,
      hidden_size=self.hidden_size,
      device=self.device).to(self.device)
    
    loss_fn = nn.MSELoss().to(self.device)
    optimizer = optim.Adam(model.parameters(), lr=self.lr)
    
    print("TRAIN START")
    for epoch in range(1, self.n_epochs+1):
      model.train()
      train_loss = []
      for batch in self.train_dataloader:
        X, y = batch
        X = X.float().to(self.device)
        y = y.float().to(self.device)
        outputs = model.forward(X)
        optimizer.zero_grad()
        loss = loss_fn(outputs, y)
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
      
      model.eval()
      with torch.no_grad():
        val_loss = []
        for batch in self.val_dataloader:
          X, y = batch
          # numpy 기본 자료형은 float64이므로 float32에 매칭시켜줘야함
          X = X.float().to(self.device)
          y = y.float().to(self.device)
          outputs = model.forward(X)
          loss = loss_fn(outputs, y)
          val_loss.append(loss.item())
      
      
      if epoch % 10 == 0:
        # 데이터에 nan값이 있는 경우 loss가 계속 nan값이 나옴
        print(f"Epoch: {epoch},\
          train loss: {sum(train_loss)/len(train_loss):.5f},\
          val loss: {sum(val_loss)/len(val_loss):.5f}")
        
    print("TRAIN FIN")

    return model
  
      
  def plot(self, model):
    model.eval()
    with torch.no_grad():
      X, y = next(iter(self.val_dataloader))
      X = X.float().to(self.device)
      y = y.float().to(self.device)
      output = model.forward(X)
      print(output.shape)
      print(y.shape)
    y = y.cpu().numpy()
    output = output.cpu().numpy()
    plt.plot(output[0].squeeze(0))

In [ ]:
trainer = Trainer(path="./dataset/CUSTOM_v0/공주.csv")
trainer.set_up()

In [ ]:
mdl = trainer.train()
torch.save(mdl, "lstm.pth")